In [ ]:
!pip install openpyxl
!pip install huggingface_hub
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.75.0
    Uninstalling openai-1.75.0:
      Successfully uninstalled openai-1.75.0


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM,pipeline
import torch
import numpy as np
import json
import pandas as pd
from huggingface_hub import login
import os
import pandas as pd
import openai
import string
import ast
import re

In [ ]:
login()
openai.api_key = 'Your-api'
# comment the below line if you use original OPENAI api
openai.api_base = 'https://api.avalai.ir/v1'

In [ ]:
# @title model setting
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/llama-3-meerkat-8b-v1.0")
model = AutoModelForCausalLM.from_pretrained("dmis-lab/llama-3-meerkat-8b-v1.0",torch_dtype=torch.bfloat16).to(device)

translator_tokenizer = AutoTokenizer.from_pretrained("CohereForAI/aya-expanse-8b")
translator_model = AutoModelForCausalLM.from_pretrained(
    "CohereForAI/aya-expanse-8b",
    torch_dtype=torch.bfloat16
).to(device)

translator_model.eval()
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_

In [ ]:
# @title Utility


def translate(text,src,des):
    prompt = "you are an expert translator in medical domain please translate the given" + src + "question to" + des + "please do not provide any additional information or explanations" + '\n' + "text: "
    pipe = pipeline("text-generation", model=translator_model, tokenizer=translator_tokenizer)
    pipe_output = pipe([{"role": "user", "content": f"{prompt+text}"}],
                       max_new_tokens=1024,
                       eos_token_id=[translator_tokenizer.eos_token_id],
                       do_sample=False)
    return pipe_output[0]["generated_text"][-1]["content"]

def get_AI_answer(Prompt,Question):
    content = translate(Prompt + '\n' + Question,"persian","english")
    messages = [{"role": "user", "content": f"{content}"}]
    input_ids = tokenizer.apply_chat_template(
                messages,
                add_generation_prompt=True,
                return_tensors="pt"
                ).to(model.device)

    terminators = [
                tokenizer.eos_token_id,
                tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
              input_ids,
              max_new_tokens=1024,
              eos_token_id=terminators,
              do_sample=False,
              temperature=0.0,)
    response = outputs[0][input_ids.shape[-1]:]
    return translate(tokenizer.decode(response, skip_special_tokens=True),"english","persian")

def append_record_to_excel(file_path,question,answer,source,AI_answer):
    new_record = {
        'Question': question,
        'answer': answer,
        'Sources': source,
        'AI_answer': AI_answer
    }
    new_record_df = pd.DataFrame([new_record])
    try:
        existing_df = pd.read_excel(file_path)
        updated_df = pd.concat([existing_df, new_record_df], ignore_index=True)
    except FileNotFoundError:
        updated_df = new_record_df

    updated_df.to_excel(file_path, index=False)


In [ ]:
df = pd.read_excel('test.xlsx')
Prompt = "شما یک پزشک حاذق هستید و در حوزه پزشکی دانش بسیار بالایی دارید لطفا هر آنچه در مورد پرسش مطرح شده میدانید بگویید"+'\n'+'پرسش:'
i = 0
for index,row in df.iterrows():
        question = row['Question']
        answer = row['Answer']
        source = row['Source']
        AI_answer = get_AI_answer(Prompt,question)
        file_path = 'fa_pipeline_result.xlsx'
        append_record_to_excel(file_path,question,answer,source,AI_answer)
        i += 1
        print(i)
        print('============================================================')


Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Device set to use cuda:0
Device set to use cuda:0


1596


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Device set to use cuda:0
Device set to use cuda:0


1597


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Device set to use cuda:0
Device set to use cuda:0


...

1998


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Device set to use cuda:0
Device set to use cuda:0


1999


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Device set to use cuda:0


2000
